In [ ]:
"""
VirtuWill - Apple Vision OCR (macOS)

Recursively OCRs images in a folder using Apple's Vision framework and writes:
1) A .txt file per image (mirrors input folder structure)
2) A JSONL manifest with metadata for later fine-tuning

Requirements (in your conda env):
  python -m pip install pyobjc

Recommended:
  conda install -y jupyterlab
"""

In [7]:
# === VirtuWill Environment Guard ===
import sys
import os
from pathlib import Path


EXPECTED_ENV_NAME = "virtuwill"

def fail(msg):
    raise RuntimeError(f"❌ ENVIRONMENT ERROR: {msg}")

# 1. Check sys.executable path
exe = sys.executable
if EXPECTED_ENV_NAME not in exe:
    fail(
        f"Notebook is NOT running in '{EXPECTED_ENV_NAME}'.\n"
        f"Current Python: {exe}\n\n"
        "Fix:\n"
        "  1) Kernel → Change Kernel → Python (virtuwill)\n"
        "  2) Restart kernel\n"
    )

# 2. Check conda environment variable (extra safety)
conda_env = os.environ.get("CONDA_DEFAULT_ENV")
if conda_env != EXPECTED_ENV_NAME:
    fail(
        f"CONDA_DEFAULT_ENV='{conda_env}' (expected '{EXPECTED_ENV_NAME}').\n"
        "This usually means Jupyter was launched from the wrong environment."
    )

print(f"✅ VirtuWill environment verified\nPython: {exe}")

print("📍 Python executable:")
print(sys.executable)

print("\n🐍 Python version:")
print(sys.version)

print("\n📦 Conda environment:")
print(os.environ.get("CONDA_DEFAULT_ENV"))

print("\nCurrent working directory:")
cwd = Path.cwd()
print(" ", cwd)


✅ VirtuWill environment verified
Python: /opt/anaconda3/envs/virtuwill/bin/python
📍 Python executable:
/opt/anaconda3/envs/virtuwill/bin/python

🐍 Python version:
3.12.12 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 20:07:49) [Clang 20.1.8 ]

📦 Conda environment:
virtuwill

Current working directory:
  /Users/wyoste/Documents/virtuwill


In [9]:
#!/usr/bin/env python3


from __future__ import annotations

import argparse
import json
import time
from dataclasses import dataclass
from typing import Iterable, Optional, Tuple

import objc
from Foundation import NSURL, NSAutoreleasePool, NSError
from Vision import VNImageRequestHandler, VNRecognizeTextRequest
from Quartz import CGImageSourceCreateWithURL, CGImageSourceCreateImageAtIndex


SUPPORTED_EXTS = {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".heic", ".webp"}


@dataclass
class OcrResult:
    source_image: str
    output_txt: str
    language: str
    recognition_level: str
    text: str
    num_lines: int
    elapsed_ms: int
    error: Optional[str] = None


def load_cgimage(image_path: Path):
    """
    Load an image into a CGImage using Quartz/ImageIO.
    Avoids PIL and generally works well for JPEG/PNG/HEIC on macOS.
    """
    url = NSURL.fileURLWithPath_(str(image_path))
    src = CGImageSourceCreateWithURL(url, None)
    if src is None:
        raise RuntimeError(f"Could not create image source for: {image_path}")
    cgimg = CGImageSourceCreateImageAtIndex(src, 0, None)
    if cgimg is None:
        raise RuntimeError(f"Could not create CGImage for: {image_path}")
    return cgimg


def ocr_apple_vision(
    image_path: Path,
    language: str = "en-US",
    accurate: bool = True,
) -> Tuple[str, int]:
    """
    OCR an image with Apple Vision.
    Returns (text, num_lines).
    """
    cgimg = load_cgimage(image_path)

    out = {"text": ""}

    def completion_handler(request, error):
        if error is not None:
            # error is an NSError (ObjC)
            raise RuntimeError(f"Vision error: {error}")

        observations = request.results() or []
        lines = []
        for obs in observations:
            candidates = obs.topCandidates_(1)
            if candidates and len(candidates) > 0:
                lines.append(candidates[0].string())
        out["text"] = "\n".join(lines)

    req = VNRecognizeTextRequest.alloc().initWithCompletionHandler_(completion_handler)

    # VNRequestTextRecognitionLevelAccurate = 1, Fast = 0
    req.setRecognitionLevel_(1 if accurate else 0)
    req.setUsesLanguageCorrection_(True)
    req.setRecognitionLanguages_([language])

    handler = VNImageRequestHandler.alloc().initWithCGImage_options_(cgimg, None)

    ok = handler.performRequests_error_([req], objc.nil)
    if not ok:
        raise RuntimeError("Failed to perform Vision request (performRequests_error_ returned false)")

    text = out["text"]
    num_lines = 0 if not text.strip() else text.count("\n") + 1
    return text, num_lines


def iter_images(root: Path) -> Iterable[Path]:
    for p in root.rglob("*"):
        if p.is_file() and p.suffix.lower() in SUPPORTED_EXTS:
            yield p


def relative_output_txt(input_root: Path, output_root: Path, image_path: Path) -> Path:
    rel = image_path.relative_to(input_root)
    # mirror structure, but replace extension with .txt
    return (output_root / rel).with_suffix(".txt")


def ensure_parent(path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)


def write_text(path: Path, text: str) -> None:
    ensure_parent(path)
    path.write_text(text, encoding="utf-8")


def append_jsonl(path: Path, record: dict) -> None:
    ensure_parent(path)
    with path.open("a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")


def main():
    parser = argparse.ArgumentParser(
        description="VirtuWill: OCR journal images locally using Apple Vision (macOS)."
    )
    parser.add_argument(
        "--input",
        default="/Users/wyoste/Pictures/Journals POC",
        help="Root folder containing images (default: /Users/wyoste/Pictures/Journals POC)",
    )
    parser.add_argument(
        "--output",
        default="/Users/wyoste/Pictures/Journals POC/VirtuWill_OCR_TXT",
        help="Output root folder for OCR .txt files",
    )
    parser.add_argument(
        "--manifest",
        default="/Users/wyoste/Pictures/Journals POC/VirtuWill_OCR_TXT/manifest.jsonl",
        help="JSONL manifest path for results/metadata",
    )
    parser.add_argument(
        "--language",
        default="en-US",
        help="Vision OCR language code (default: en-US)",
    )
    parser.add_argument(
        "--fast",
        action="store_true",
        help="Use faster, less accurate recognition (default is Accurate)",
    )
    parser.add_argument(
        "--overwrite",
        action="store_true",
        help="Overwrite existing .txt outputs (default: skip if exists)",
    )
    parser.add_argument(
        "--limit",
        type=int,
        default=0,
        help="Process only first N images (0 = no limit)",
    )
    # Use parse_known_args to tolerate Jupyter/ipykernel injected flags (e.g. -f kernel.json)
    args, _ = parser.parse_known_args()

    input_root = Path(args.input).expanduser()
    output_root = Path(args.output).expanduser()
    manifest_path = Path(args.manifest).expanduser()

    if not input_root.exists():
        print(f"ERROR: Input path does not exist: {input_root}", file=sys.stderr)
        sys.exit(1)

    images = list(iter_images(input_root))
    if args.limit and args.limit > 0:
        images = images[: args.limit]

    if not images:
        print(f"No supported images found under: {input_root}")
        return

    recognition_level = "FAST" if args.fast else "ACCURATE"
    print(f"Input:    {input_root}")
    print(f"Output:   {output_root}")
    print(f"Manifest: {manifest_path}")
    print(f"Images:   {len(images)}")
    print(f"Lang:     {args.language}")
    print(f"Level:    {recognition_level}")
    print("-" * 60)

    failures = 0
    started = time.time()

    # Helps avoid ObjC memory growth during long runs
    pool = NSAutoreleasePool.alloc().init()
    try:
        for i, img_path in enumerate(images, start=1):
            rel = img_path.relative_to(input_root)
            out_txt = relative_output_txt(input_root, output_root, img_path)

            if out_txt.exists() and not args.overwrite:
                print(f"[{i}/{len(images)}] SKIP (exists): {rel}")
                # Still log a manifest record as skipped (optional)
                append_jsonl(manifest_path, {
                    "source_image": str(img_path),
                    "output_txt": str(out_txt),
                    "status": "skipped_exists",
                    "language": args.language,
                    "recognition_level": recognition_level,
                    "timestamp": int(time.time()),
                })
                continue

            t0 = time.time()
            try:
                text, num_lines = ocr_apple_vision(
                    img_path,
                    language=args.language,
                    accurate=(not args.fast),
                )
                elapsed_ms = int((time.time() - t0) * 1000)

                write_text(out_txt, text)

                append_jsonl(manifest_path, {
                    "source_image": str(img_path),
                    "output_txt": str(out_txt),
                    "status": "ok",
                    "language": args.language,
                    "recognition_level": recognition_level,
                    "num_lines": num_lines,
                    "elapsed_ms": elapsed_ms,
                    "timestamp": int(time.time()),
                })

                print(f"[{i}/{len(images)}] OK  ({elapsed_ms} ms, {num_lines} lines): {rel}")

            except Exception as e:
                failures += 1
                elapsed_ms = int((time.time() - t0) * 1000)

                append_jsonl(manifest_path, {
                    "source_image": str(img_path),
                    "output_txt": str(out_txt),
                    "status": "error",
                    "language": args.language,
                    "recognition_level": recognition_level,
                    "elapsed_ms": elapsed_ms,
                    "error": str(e),
                    "timestamp": int(time.time()),
                })

                print(f"[{i}/{len(images)}] FAIL ({elapsed_ms} ms): {rel} :: {e}", file=sys.stderr)

    finally:
        del pool

    total_s = int(time.time() - started)
    print("-" * 60)
    print(f"Done. Failures: {failures}. Total time: {total_s}s")
    print(f"OCR text root: {output_root}")
    print(f"Manifest:      {manifest_path}")


if __name__ == "__main__":
    main()


Input:    /Users/wyoste/Pictures/Journals POC
Output:   /Users/wyoste/Pictures/Journals POC/VirtuWill_OCR_TXT
Manifest: /Users/wyoste/Pictures/Journals POC/VirtuWill_OCR_TXT/manifest.jsonl
Images:   12
Lang:     en-US
Level:    ACCURATE
------------------------------------------------------------
[1/12] OK  (235 ms, 9 lines): FF08A95B-48AE-427E-93B8-298E92C1C6A0_4_5005_c.jpeg
[2/12] OK  (45 ms, 27 lines): FE8F1C83-57B2-41A9-B235-97BD5FAC8340_1_102_o.jpeg
[3/12] OK  (87 ms, 30 lines): FDCD7133-50D4-4577-8AF1-709BD7691351_1_102_o.jpeg
[4/12] OK  (22 ms, 8 lines): FF39CC49-083F-463F-B883-C65921E11C9C_1_102_o.jpeg
[5/12] OK  (13 ms, 6 lines): FD1B2179-4444-460B-A56E-030404180C85_4_5005_c.jpeg
[6/12] OK  (75 ms, 38 lines): FD487870-9160-49AA-9212-0BC01C4ED8BB_1_102_o.jpeg
[7/12] OK  (91 ms, 31 lines): FEAA6AB4-2B21-461B-AC12-917A85BEB1D1_1_102_o.jpeg
[8/12] OK  (64 ms, 13 lines): FD7461A8-9F65-406A-A4E1-188B964847C2_1_102_o.jpeg
[9/12] OK  (88 ms, 25 lines): FEC331F3-B258-4BC4-B41A-FBB6998A